In [5]:
# General
# %pip install numpy
# %pip install scipy
# %pip install matplotlib
# %pip install soundfile
import numpy as np

# Signal Processing
from scipy import signal
from scipy.fftpack import fft, ifft

# Audio IO
from IPython.display import Audio 
import soundfile as sf

# Plotting
import matplotlib.pyplot as plt

In [ ]:
# x, Fs = sf.read("bad_audio.wav")

# Audio(x, rate = Fs)

In [ ]:
# implement 2nd order peaking filter to remove 573Hz tone here
x, Fs = sf.read("bad_audio.wav")
def second_order_peak(audio_arr, cutoff, bandwidth, sr, G):
    return_audio = np.zeros(len(audio_arr))
    wC = 2 * cutoff/sr
    wB = 2 * bandwidth/sr   
    v0 = 10 ** (G / 20)
    h0 = v0 - 1
    c = None
    if G >= 0:
        c = (np.tan(np.pi * wB/2)-1) / (np.tan(np.pi * wB/2)+1)
    else:
        c = (np.tan(np.pi * wB/2)-v0) / (np.tan(np.pi * wB/2)+v0)
    d = -np.cos(np.pi * wC)
    xH = [0,0]
    for i in range(len(audio_arr)):
        xH_new = audio_arr[i] - d*(1-c) * xH[0] + c*xH[1]
        ap_y = -c * xH_new + d*(1-c) * xH[0] + xH[1]
        xH = [xH_new, xH[0]]
        return_audio[i] = 0.5 * h0 * (audio_arr[i] - ap_y) + audio_arr[i]
    return return_audio

hp_audio = second_order_peak(x, 570, 1, Fs, -80)
# hp_audio *= 1000.0
Audio(hp_audio, rate = Fs)


In [ ]:
# implement 2nd order high shelf filter to remove >14kHz noise here
def high_shelf(audio_arr, cutoff, sr, G):
    return_audio = np.zeros(len(audio_arr))
    wC = 2 * cutoff/sr
    v0 = 10 ** (G / 20)
    K = np.tan(np.pi * cutoff/sr)
    b0 = (v0(1 + np.sqrt(2) * K + K ** 2)) / (1 + np.sqrt(2) * K + v0 * K ** 2)
    b1 = 2 * (v0(K ** 2 - 1)) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    b2 = (v0(1 - np.sqrt(2) * K + K ** 2)) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    a1 = (2 * (v0 * K ** 2 - 1)) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    a2 = (1 - np.sqrt(2 * v0) * K + v0 * K ** 2) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    return return_audio


In [ ]:
# Audio(y, rate = Fs)